<a href="https://colab.research.google.com/github/NadaAdelMousa/Pose-Estimation/blob/main/Pose_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nada Adel AbdEl-azem 

---


Smart -- Group_1

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 330 kB/s 


In [2]:
import mediapipe
import cv2
import scipy.io as sio
import numpy as np
import os
from math import cos, sin
from google.colab.patches import cv2_imshow

In [3]:
if os.path.isfile('/content/AFLW2000-3D.zip') == False:
  !gdown --id  1fP3zvSCYjll_o_m7S12nvQLZ9MnsEoap
  !unzip /content/AFLW2000-3D.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1fP3zvSCYjll_o_m7S12nvQLZ9MnsEoap
To: /content/AFLW2000-3D.zip
100% 87.4M/87.4M [00:00<00:00, 228MB/s]
Archive:  /content/AFLW2000-3D.zip
   creating: AFLW2000/
   creating: AFLW2000/Code/
  inflating: AFLW2000/Code/DrawSolidHead.m  
  inflating: AFLW2000/Code/DrawTextureHead.m  
  inflating: AFLW2000/Code/main_show_without_BFM.m  
  inflating: AFLW2000/Code/main_show_with_BFM.m  
   creating: AFLW2000/Code/Mex/
  inflating: AFLW2000/Code/Mex/compile.m  
  inflating: AFLW2000/Code/Mex/Tnorm_Vnorm.h  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.cpp  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.mexw64  
   creating: AFLW2000/Code/ModelGeneration/
  inflating: AFLW2000/Code/ModelGeneration/ModelGenerate.m  
  inflatin

In [4]:
Data = np.zeros((1853, 937))
pitch_labels = np.zeros((1853, 1))
yaw_labels = np.zeros((1853, 1))
roll_labels = np.zeros((1853, 1))
data_point = 0

In [5]:
faceModule = mediapipe.solutions.face_mesh

faces = faceModule.FaceMesh(static_image_mode=True)

In [6]:
#Extract Landmarks from images, labels from .mat files
for i in range(5000):
    img_path = 'AFLW2000/image' + str(i).zfill(5) + '.jpg'
    if os.path.isfile(img_path):
        image = cv2.imread(img_path)
        mat_file = sio.loadmat('AFLW2000/image' + str(i).zfill(5) + '.mat')
        pose_para = mat_file["Pose_Para"][0][:3]
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks is not None:
            print(img_path)
            pitch_labels[data_point] = pose_para[0]
            yaw_labels[data_point]   = pose_para[1]
            roll_labels[data_point]  = pose_para[2]
            center_point_x = results.multi_face_landmarks[0].landmark[4].x
            center_point_y = results.multi_face_landmarks[0].landmark[4].y
            Const_distance_x = results.multi_face_landmarks[0].landmark[4].x - results.multi_face_landmarks[0].landmark[60].x
            Const_distance_y = results.multi_face_landmarks[0].landmark[4].y - results.multi_face_landmarks[0].landmark[60].y
            Const_distance = np.sqrt(Const_distance_x**2 + Const_distance_y**2)
            for face in results.multi_face_landmarks:
              feature=0;
              Data[data_point][feature]   =  i
              feature = feature + 1
              for landmark in face.landmark:
                Data[data_point][feature]   =  (landmark.x - center_point_x) /Const_distance
                Data[data_point][feature+1] =  (landmark.y - center_point_y) /Const_distance
                feature = feature + 2
            data_point = data_point + 1

AFLW2000/image00002.jpg
AFLW2000/image00004.jpg
AFLW2000/image00006.jpg
AFLW2000/image00008.jpg
AFLW2000/image00013.jpg
AFLW2000/image00014.jpg
AFLW2000/image00019.jpg
AFLW2000/image00020.jpg
AFLW2000/image00021.jpg
AFLW2000/image00022.jpg
AFLW2000/image00023.jpg
AFLW2000/image00026.jpg
AFLW2000/image00028.jpg
AFLW2000/image00035.jpg
AFLW2000/image00039.jpg
AFLW2000/image00040.jpg
AFLW2000/image00041.jpg
AFLW2000/image00042.jpg
AFLW2000/image00043.jpg
AFLW2000/image00044.jpg
AFLW2000/image00045.jpg
AFLW2000/image00046.jpg
AFLW2000/image00047.jpg
AFLW2000/image00048.jpg
AFLW2000/image00049.jpg
AFLW2000/image00050.jpg
AFLW2000/image00051.jpg
AFLW2000/image00052.jpg
AFLW2000/image00053.jpg
AFLW2000/image00054.jpg
AFLW2000/image00055.jpg
AFLW2000/image00056.jpg
AFLW2000/image00057.jpg
AFLW2000/image00058.jpg
AFLW2000/image00059.jpg
AFLW2000/image00060.jpg
AFLW2000/image00061.jpg
AFLW2000/image00062.jpg
AFLW2000/image00063.jpg
AFLW2000/image00064.jpg
AFLW2000/image00065.jpg
AFLW2000/image00

In [7]:
#Fitting the data
from sklearn.ensemble import RandomForestRegressor

Model_pitch = RandomForestRegressor(max_depth=30, random_state=0).fit(Data[:,1:], np.ravel(pitch_labels))
Model_yaw   = RandomForestRegressor(max_depth=30, random_state=0).fit(Data[:,1:], np.ravel(yaw_labels))
Model_roll =  RandomForestRegressor(max_depth=30, random_state=0).fit(Data[:,1:], np.ravel(roll_labels))

print(Model_pitch.score(Data[:,1:], pitch_labels))
print(Model_yaw.score(Data[:,1:], yaw_labels))
print(Model_roll.score(Data[:,1:], roll_labels))

0.7927149152955579
0.9802726435822798
0.9055029774528253


In [8]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 60):
    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
      results = faces.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
      if results.multi_face_landmarks is not None:
        tdx = int(results.multi_face_landmarks[0].landmark[5].x * img.shape[1])
        tdy = int(results.multi_face_landmarks[0].landmark[5].y * img.shape[0])
      else:
        return img

       # height, width = img.shape[:2]
       # tdx = width / 2
       # tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),2)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),2)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [9]:
cap = cv2.VideoCapture('test_video.mp4')

if (cap.isOpened()== False): 
  print("Error opening video file")

int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

595

In [10]:
faceModule = mediapipe.solutions.face_mesh

faces = faceModule.FaceMesh(static_image_mode=True)

In [11]:
test_Data = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),936))
data_point = 0
feature = 0

In [12]:
#Extract Landmarks from test video
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    results = faces.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks is not None:
      center_point_x = results.multi_face_landmarks[0].landmark[4].x
      center_point_y = results.multi_face_landmarks[0].landmark[4].y
      Const_distance_x = results.multi_face_landmarks[0].landmark[4].x - results.multi_face_landmarks[0].landmark[60].x
      Const_distance_y = results.multi_face_landmarks[0].landmark[4].y - results.multi_face_landmarks[0].landmark[60].y
      Const_distance = np.sqrt(Const_distance_x**2 + Const_distance_y**2)
      print(data_point)
      for face in results.multi_face_landmarks:
        feature=0
        for landmark in face.landmark:
          test_Data[data_point][feature]   =  (landmark.x - center_point_x) /Const_distance
          test_Data[data_point][feature+1] =  (landmark.y - center_point_y) /Const_distance
          feature = feature + 2
      data_point = data_point + 1
    else:
      test_Data[data_point]   =  np.zeros(936)
      data_point = data_point + 1
      print(data_point, '  0')
  else:
    break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
cap = cv2.VideoCapture('test_video.mp4')

if (cap.isOpened()== False): 
  print("Error opening video file")

int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

595

In [17]:
#Use the models on a test video and save the output video
img_array = []
test_image = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
      frame = draw_axis(frame,
                      Model_pitch.predict(test_Data[test_image,:].reshape(1,-1)),
                      Model_yaw.predict(test_Data[test_image,:].reshape(1,-1)),
                      Model_roll.predict(test_Data[test_image,:].reshape(1,-1)))

      img_array.append(frame)
      height, width, layers = frame.shape
      size  = (width,height)
      test_image = test_image + 1
    else:
      break

out = cv2.VideoWriter('test_video_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
    print(i)
out.release()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
cap.release()
cv2.destroyAllWindows()
test_Data.shape

(595, 936)